# Distance (Space) Functions:

In [14]:
def calcular_diferenca(df, bodypart="center", sig=6):
    """
    Esta função calcula as colunas de diferença (delta_x e delta_y) entre coordenadas consecutivas de uma parte do corpo.

    Parâmetros:
    - df: DataFrame contendo os dados a serem processados.
    - bodypart: Nome da parte do corpo para a qual as coordenadas estão sendo calculadas.

    Retorna:
    - DataFrame com as colunas de diferença adicionadas.
    """
    import pandas as pd
    from scipy.ndimage import gaussian_filter1d

    df_delta = df.copy()

    df_delta['delta_x'] = df[f'{bodypart}_x'].diff()
    df_delta['delta_y'] = df[f'{bodypart}_y'].diff()

    # Preencher os valores NaN com zeros ou valores iniciais
    df_delta['delta_x'].fillna(0, inplace=True)
    df_delta['delta_y'].fillna(0, inplace=True)

    df['distance_x'] = gaussian_filter1d(df_delta['delta_x'], sigma=sig)
    df['distance_y'] = gaussian_filter1d(df_delta['delta_y'], sigma=sig)

    return df

In [15]:
def calcular_distancia_pixels(df):
    """
    Esta função calcula a coluna de distância euclidiana entre pontos consecutivos em um DataFrame.

    Parâmetros:
    - df: DataFrame contendo as coordenadas e as colunas de diferença (delta_x e delta_y).

    Retorna:
    - DataFrame com a coluna de distância adicionada.
    """
    import pandas as pd
    import numpy as np

    df['distance_pixels'] = np.sqrt(df['distance_x']**2 + df['distance_y']**2)
    return df

In [16]:
def calcular_distancia_cm(df, conversion_rate):
    """
    Converts a column in DataFrame from pixels to centimeters.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the column to be converted.
    - column_name (str): The name of the column to be converted.
    - conversion_rate (float): The conversion rate from pixels to centimeters.

    Returns:
    - pd.DataFrame: The DataFrame with the new column representing distances in centimeters.
    """
    import pandas as pd
    import numpy as np

    df_copy = df.copy()
    df_copy["distance_cm"] = df_copy["distance_pixels"] / conversion_rate
    return df_copy

In [17]:
def spaces(df, conversion_rate):
    import pandas as pd
    import numpy as np

    space_pixels = calcular_distancia_pixels(df)
    result = calcular_distancia_cm(df, conversion_rate)
    return result

# FPS_video:

In [18]:
def calcular_fps(df, min):
    """
    Calcula a taxa de quadros por segundo (FPS) com base na quantidade de quadros e na duração em minutos.

    Parâmetros:
    - df (pd.DataFrame): DataFrame que represente um video
    - min (float): Duração do vídeo em minutos.

    Retorna:
    - float: Taxa de quadros por segundo (FPS).
    """
    fps = len(df)/(min*60)
    return fps

# Time Functions:

In [19]:
def times(df):
    """
    Esta função calcula as colunas de tempo em segundos e minutos com base nos dados do DataFrame e no DataFrame de fps.

    Parâmetros:
    - df: DataFrame contendo os dados a serem processados.
    - fps_df: DataFrame contendo as informações de fps.

    Retorna:
    - DataFrame com as colunas de tempo adicionadas.
    """
    import pandas as pd

    df = df.reset_index(drop=True)

    list_times = []
    result_df = pd.DataFrame()

    # Obter todas as combinações únicas de 'rat' e 'day'
    combinacoes_unicas = df[['rat', 'day']].drop_duplicates()

    # Iterar sobre as combinações únicas
    for _, combinacao in combinacoes_unicas.iterrows():
        rat_unico = combinacao['rat']
        day_unico = combinacao['day']

        temp_df = df[(df['rat'] == rat_unico) & (df['day'] == day_unico)].copy()

        fps_value = 15

        # Calcula as colunas de tempo em segundos e minutos
        temp_df['tempo_segundos'] = temp_df['frame'] / fps_value
        temp_df['tempo_minutos'] = temp_df['tempo_segundos'] / 60

        # Adiciona o resultado ao DataFrame final
        result_df = pd.concat([result_df, temp_df])

    return result_df

# Speed:

In [20]:
def speed_rat(df):
  import pandas as pd

  df = df.reset_index(drop=True)

  list_times = []
  result_df = pd.DataFrame()

  # Obter todas as combinações únicas de 'rat' e 'day'
  combinacoes_unicas = df[['rat', 'day']].drop_duplicates()

  # Iterar sobre as combinações únicas
  for _, combinacao in combinacoes_unicas.iterrows():
      rat_unico = combinacao['rat']
      day_unico = combinacao['day']

      df_speed = df[(df['rat'] == rat_unico) & (df['day'] == day_unico)].copy()

      min = df_speed[f'tempo_minutos'].max()

      fps_value = 15

      df_speed[f'instantaneous_speed'] = df_speed[f'distance_cm']/ (1/fps_value)

      # Adiciona o resultado ao DataFrame final
      result_df = pd.concat([result_df, df_speed])

  return result_df

#  Import Base Libs:

In [21]:
import pandas as pd  # Manipulação de dados tabulares (DataFrames)
import numpy as np  # Manipulação eficiente de arrays e operações matemáticas
import os  # Manipulação de sistema de arquivos

import os
import glob

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tests:

In [22]:
directory = "/content/drive/MyDrive/PPGNeuro/area_Exp35/Exp 35 - rodada 2 - analisados/result_artemis/dataset_select.csv"
df_coords = pd.read_csv(directory)
df_coords

,frame,id_amostra,rat,experimental_context,day,center_b_x,center_b_y,center_b_likelihood
0,0,CCM247-TR-Contexto2,CCM247,2,4,886.874023,96.242256,0.990015
1,1,CCM247-TR-Contexto2,CCM247,2,4,887.161743,96.214500,0.990392
2,2,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359
3,3,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359
4,4,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359
...,...,...,...,...,...,...,...,...
1197015,17997,CCM271-HAB2-Contexto1,CCM271,1,2,403.220032,643.497559,0.960405
1197016,17998,CCM271-HAB2-Contexto1,CCM271,1,2,404.952240,643.319763,0.960552
1197017,17999,CCM271-HAB2-Contexto1,CCM271,1,2,404.614166,641.280334,0.955707
1197018,18000,CCM271-HAB2-Contexto1,CCM271,1,2,404.647156,641.285034,0.944732


In [23]:
regiao = "center_b"
min_likelihood = 0.6
taxa_conversao = 10.91

In [24]:
df = calcular_diferenca(df_coords, regiao)
df

,frame,id_amostra,rat,experimental_context,day,center_b_x,center_b_y,center_b_likelihood,distance_x,distance_y
0,0,CCM247-TR-Contexto2,CCM247,2,4,886.874023,96.242256,0.990015,-0.189358,0.047759
1,1,CCM247-TR-Contexto2,CCM247,2,4,887.161743,96.214500,0.990392,-0.190546,0.044808
2,2,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.191308,0.040720
3,3,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.188941,0.039072
4,4,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.179237,0.045740
...,...,...,...,...,...,...,...,...,...,...
1197015,17997,CCM271-HAB2-Contexto1,CCM271,1,2,403.220032,643.497559,0.960405,0.014725,-0.064466
1197016,17998,CCM271-HAB2-Contexto1,CCM271,1,2,404.952240,643.319763,0.960552,0.012066,-0.065594
1197017,17999,CCM271-HAB2-Contexto1,CCM271,1,2,404.614166,641.280334,0.955707,0.009257,-0.066048
1197018,18000,CCM271-HAB2-Contexto1,CCM271,1,2,404.647156,641.285034,0.944732,0.006972,-0.066125


In [25]:
df["day"].unique()

array([4, 5, 3, 1, 2])

In [26]:
df_result = times(df)
df_result

,frame,id_amostra,rat,experimental_context,day,center_b_x,center_b_y,center_b_likelihood,distance_x,distance_y,tempo_segundos,tempo_minutos
0,0,CCM247-TR-Contexto2,CCM247,2,4,886.874023,96.242256,0.990015,-0.189358,0.047759,0.000000,0.000000
1,1,CCM247-TR-Contexto2,CCM247,2,4,887.161743,96.214500,0.990392,-0.190546,0.044808,0.066667,0.001111
2,2,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.191308,0.040720,0.133333,0.002222
3,3,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.188941,0.039072,0.200000,0.003333
4,4,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.179237,0.045740,0.266667,0.004444
...,...,...,...,...,...,...,...,...,...,...,...,...
1197015,17997,CCM271-HAB2-Contexto1,CCM271,1,2,403.220032,643.497559,0.960405,0.014725,-0.064466,1199.800000,19.996667
1197016,17998,CCM271-HAB2-Contexto1,CCM271,1,2,404.952240,643.319763,0.960552,0.012066,-0.065594,1199.866667,19.997778
1197017,17999,CCM271-HAB2-Contexto1,CCM271,1,2,404.614166,641.280334,0.955707,0.009257,-0.066048,1199.933333,19.998889
1197018,18000,CCM271-HAB2-Contexto1,CCM271,1,2,404.647156,641.285034,0.944732,0.006972,-0.066125,1200.000000,20.000000


In [27]:
df_end = spaces(df_result, conversion_rate=taxa_conversao)
df_end

,frame,id_amostra,rat,experimental_context,day,center_b_x,center_b_y,center_b_likelihood,distance_x,distance_y,tempo_segundos,tempo_minutos,distance_pixels,distance_cm
0,0,CCM247-TR-Contexto2,CCM247,2,4,886.874023,96.242256,0.990015,-0.189358,0.047759,0.000000,0.000000,0.195287,0.017900
1,1,CCM247-TR-Contexto2,CCM247,2,4,887.161743,96.214500,0.990392,-0.190546,0.044808,0.066667,0.001111,0.195743,0.017942
2,2,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.191308,0.040720,0.133333,0.002222,0.195594,0.017928
3,3,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.188941,0.039072,0.200000,0.003333,0.192939,0.017685
4,4,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.179237,0.045740,0.266667,0.004444,0.184981,0.016955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197015,17997,CCM271-HAB2-Contexto1,CCM271,1,2,403.220032,643.497559,0.960405,0.014725,-0.064466,1199.800000,19.996667,0.066127,0.006061
1197016,17998,CCM271-HAB2-Contexto1,CCM271,1,2,404.952240,643.319763,0.960552,0.012066,-0.065594,1199.866667,19.997778,0.066695,0.006113
1197017,17999,CCM271-HAB2-Contexto1,CCM271,1,2,404.614166,641.280334,0.955707,0.009257,-0.066048,1199.933333,19.998889,0.066694,0.006113
1197018,18000,CCM271-HAB2-Contexto1,CCM271,1,2,404.647156,641.285034,0.944732,0.006972,-0.066125,1200.000000,20.000000,0.066491,0.006095


In [28]:
df_ultra = speed_rat(df_end)
df_ultra

,frame,id_amostra,rat,experimental_context,day,center_b_x,center_b_y,center_b_likelihood,distance_x,distance_y,tempo_segundos,tempo_minutos,distance_pixels,distance_cm,instantaneous_speed
0,0,CCM247-TR-Contexto2,CCM247,2,4,886.874023,96.242256,0.990015,-0.189358,0.047759,0.000000,0.000000,0.195287,0.017900,0.268498
1,1,CCM247-TR-Contexto2,CCM247,2,4,887.161743,96.214500,0.990392,-0.190546,0.044808,0.066667,0.001111,0.195743,0.017942,0.269124
2,2,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.191308,0.040720,0.133333,0.002222,0.195594,0.017928,0.268919
3,3,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.188941,0.039072,0.200000,0.003333,0.192939,0.017685,0.265269
4,4,CCM247-TR-Contexto2,CCM247,2,4,887.154846,96.205910,0.990359,-0.179237,0.045740,0.266667,0.004444,0.184981,0.016955,0.254328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197015,17997,CCM271-HAB2-Contexto1,CCM271,1,2,403.220032,643.497559,0.960405,0.014725,-0.064466,1199.800000,19.996667,0.066127,0.006061,0.090917
1197016,17998,CCM271-HAB2-Contexto1,CCM271,1,2,404.952240,643.319763,0.960552,0.012066,-0.065594,1199.866667,19.997778,0.066695,0.006113,0.091698
1197017,17999,CCM271-HAB2-Contexto1,CCM271,1,2,404.614166,641.280334,0.955707,0.009257,-0.066048,1199.933333,19.998889,0.066694,0.006113,0.091696
1197018,18000,CCM271-HAB2-Contexto1,CCM271,1,2,404.647156,641.285034,0.944732,0.006972,-0.066125,1200.000000,20.000000,0.066491,0.006095,0.091418


In [29]:
# Agrupar por 'rat' e 'day' e calcular a soma das colunas desejadas
result_sum = df_ultra.groupby(['rat', 'day', "experimental_context"])[['distance_pixels', 'distance_cm']].sum().reset_index()

# Agrupar por 'rat' e 'day' e calcular o máximo das colunas desejadas
result_max = df_ultra.groupby(['rat', 'day', "experimental_context"])[['tempo_segundos', 'tempo_minutos']].max().reset_index()

# Mesclar os dois DataFrames com base nas colunas 'rat' e 'day'
result = pd.merge(result_sum, result_max, on=['rat', 'day', "experimental_context"])

result['distance_m'] = result['distance_cm'] / 100

result['velocidade_media'] = result['distance_cm'] / result['tempo_segundos']

# Exibir o resultado
result

,rat,day,experimental_context,distance_pixels,distance_cm,tempo_segundos,tempo_minutos,distance_m,velocidade_media
0,CCM247,1,2,30622.308248,2806.811022,1200.000000,20.000000,28.068110,2.339009
1,CCM247,2,2,31040.035503,2845.099496,1200.000000,20.000000,28.450995,2.370916
2,CCM247,3,2,21771.109254,1995.518722,1200.066667,20.001111,19.955187,1.662840
3,CCM247,4,2,5433.553967,498.034277,90.066667,1.501111,4.980343,5.529618
4,CCM247,5,2,14227.131253,1304.045028,300.000000,5.000000,13.040450,4.346817
...,...,...,...,...,...,...,...,...,...
95,CCM273,1,2,20808.671302,1907.302594,1200.000000,20.000000,19.073026,1.589419
96,CCM273,2,2,22712.512923,2081.806867,1200.000000,20.000000,20.818069,1.734839
97,CCM273,3,2,23595.762986,2162.764710,1199.933333,19.998889,21.627647,1.802404
98,CCM273,4,2,5712.210505,523.575665,89.866667,1.497778,5.235757,5.826139


In [30]:
directory_out = "/content/drive/MyDrive/PPGNeuro/area_Exp35/Exp 35 - rodada 2 - analisados"
result.to_csv(directory_out+'/result_artemis/distance_speed.csv', index=False)
result.to_excel(directory_out+'/result_artemis/distance_speed.xlsx', index=False)